In [12]:
import pandas as pd
import tweepy

# Leer las llaves
keys = pd.read_csv("mis_llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [13]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [14]:
tweets = tweepy.Cursor(api.search,
                       q=['atletico nacional -RT'],
                       lang='es',
                       geocode='3.42158,-76.5205,1000km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [15]:
# Objeto generador
t = [tweet for tweet in tweets]

In [17]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Medellin')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,"¡ A TODO O NADA VERDOLAGAS !\n\nEl partido para la clasificación a la siguiente ronda se juega en Chile, Atlético Nacional obligado a vencer a Universidad Católica. ¡No se pierdan todas las emociones del juego escuchando @DeportesG5 \n\n¡Los esperamos en sintonía! https://t.co/yMYYryzb20",2021-05-24 21:04:12,Medellin,"Medellín, Colombia",1396935085077696513,https://twitter.com/i/web/stuatus/1396935085077696513
1,"De embajadora a escarlata.Paula Villarraga es la nueva portera del @AmericaCaliFem la bogotana de 29 años quien tuvo paso por Fortaleza, Atlético Nacional y Millonarios ya inició trabajos con el equipo y llega para pelear la titularidad del arco escarlata. Para ella muchos éxitos https://t.co/1oTIdPqH3n",2021-05-24 20:53:20,Medellin,"Valle del Cauca, Colombia",1396932350643249156,https://twitter.com/i/web/stuatus/1396932350643249156
2,"Jarlan Barrera saldría de Nacional, según, al atlético Mineiro. Pero teniendo en cuenta el pedido de un enganche del Profe Amaranto, le darías una nueva oportunidad en Junior a JB10? Espero sus opiniones!",2021-05-24 20:51:38,Medellin,"Barranquilla, Colombia",1396931920953585677,https://twitter.com/i/web/stuatus/1396931920953585677
3,🇨🇴 Se vienen dos duelos importantes para mantener el sueño de la Selección Colombia\n🤝 Rueda volverá a trabajar con jugadores que potenció y a los que dio confianza cuando estuvo en Atlético Nacional y Flamengo\n\nhttps://t.co/C7WtfRQFrA,2021-05-24 20:35:00,Medellin,Colombia,1396927734719283200,https://twitter.com/i/web/stuatus/1396927734719283200
4,"¡A vencer o morir! 💪 Con estos jugadores, 🇳🇬 Atlético Nacional (@nacionaloficial) viajó a 🇨🇱 Chile para el decisivo duelo ante la U. Católica (@Cruzados) por la 🏆 Copa Libertadores\n\nVER MÁS EN 👉https://t.co/ND9Am2J4nm https://t.co/HhxUAQAkYW",2021-05-24 20:28:48,Medellin,"Medellín, Antioquia",1396926173305782272,https://twitter.com/i/web/stuatus/1396926173305782272


In [ ]:
PUNTO 1 

In [18]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [22]:
pd.set_option('display.max_colwidth', None)

data['sent_vader'] = data['tweets'].apply(lambda val: vader.polarity_scores(val)['compound'])
data['sent_textblob'] = data['tweets'].apply(lambda val: TextBlob(val).sentiment.polarity)
data[['tweets', 'sent_vader', 'sent_textblob']].head()

data[['tweets', 'sent_vader']].sort_values('sent_vader').tail()

,tweets,sent_vader
39,"@pipe3dc @LDSoficial El cliente de un equipo de fútbol es el hincha por lo tanto tiene total derecho a reclamar si no está satisfecho con su producto, en este caso, Atlético Nacional. Lo que se hizo hoy estuvo perfecto.",0.0258
32,"¡DEMANDA!\n\nMiguel Ángel Borja demandó al Atlético Nacional por 240 mil dólares, equivalentes al 8% de su transferencia a Palmeiras que aún no se le ha pagado. \n\n#JuniorAUnClick https://t.co/scxOU3rzXp",0.1260
78,"Me parece perfecto que el presi Emilio estuviese ahí, que haya escuchado a la hinchada y se fije la idea de lo que es Atlético Nacional.",0.3182
12,"⚽️El futbolista de Atlético Nacional, Baldomero Perlaza, habló sobre su convocatoria a la Selección Colombia, siendo muy sentimentalista ante la oportunidad que tiene con la 'Tricolor' 💪🇨🇴▶https://t.co/rRbH45eLvy https://t.co/1zY0rzJjj7",0.3400
13,🇳🇬😀 Atlético Nacional contará con el regreso de Jefferson Duque para el partido contra Universidad Católica en Chile. Jarlan Barrera y Danovis Banguero también vuelven en el 'Verdolaga' ➡️➡️ https://t.co/n6oxQO7gs6 https://t.co/7YU3Ck2sPk,0.3612


In [23]:
# mensaje mas positivo
maxs = data.max()
maxs

tweets           🗣️ Sobre Vladimir Hernández:\n\nEs una opción que maneja @JuniorVAVEL, sin embargo, la edad, su alto costo en salario y su contrato vigente, sumado al hecho de que volvió a Colombia, pero a jugar con Atlético Nacional, son hechos que ponen distante la operación a que se efectúe. ✍🏽
date                                                                                                                                                                                                                                                                                    2021-05-24 21:04:12
city                                                                                                                                                                                                                                                                                               Medellin
user_location                                                                                       

In [24]:
# Mensaje mas negativo
min = data.min()
min

tweets           "El problema de Atlético Nacional es que siempre empieza los partidos jugando con 10" @Rafagol_Linares \n\n#ElSúperDebate\nhttps://t.co/vg8yCrEvyB https://t.co/7GI4o0utVf
date                                                                                                                                                                    2021-05-23 19:55:08
city                                                                                                                                                                               Medellin
user_location                                                                                                                                                                     Argentina
id                                                                                                                                                                      1396555313013272576
link                                                        